In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

In [2]:
df= pd.read_csv("data/training2.csv")
df.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,...,1.0,73.780,82.547,0.08696,10.0,0.63899,NaN,0.0,0,0
1,230033,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,18234.0,...,0.0,99.129,NaN,0.00000,13.0,0.63836,NaN,NaN,1,1
2,230034,1745.0,0.4001,0.000,0.0,0.0,NaN,2536.0,NaN,NaN,...,0.0,NaN,29.290,0.00000,1.0,1.00000,NaN,0.0,0,1
3,230035,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,...,0.0,96.272,NaN,0.15385,3.0,0.53241,0.0,0.0,1,0
4,230036,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,...,0.0,115.019,NaN,0.00000,1.0,0.92665,NaN,NaN,1,0


In [3]:
na_cols=['mvar1', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar34', 'mvar35', 'mvar36', 'mvar37', 'mvar38',
         'mvar39', 'mvar43', 'mvar45', 'mvar46']

missing_cols= ['mvar6', 'mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13', 'mvar15', 'mvar25',
               'mvar26', 'mvar27', 'mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar40', 'mvar41', 'mvar42']

null_cols= list(df.columns[df.isnull().sum() >0])

#non-missing cols == ['mvar47', 'mvar14', 'default_ind', 'application_ind']

In [4]:
df[missing_cols]= df[missing_cols].fillna(0)
df= df.fillna(df.median())
y= df['default_ind']
cols= ['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6','mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13',
       'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20','mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25',
       'mvar26', 'mvar27','mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34','mvar35', 'mvar36', 'mvar37',
       'mvar38', 'mvar39', 'mvar40', 'mvar41','mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47']
drop= ['mvar4', 'mvar5', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20', 'mvar34','mvar35', 'mvar39', 'mvar40', 'mvar41',
       'mvar45', 'mvar46']
X= df[cols]
#X= X.drop(drop, axis=1)
le= LabelEncoder()

In [5]:
test= pd.read_csv("data/Leaderboard_dataset.csv")
test= test.replace('missing', np.nan)
test= test.replace('na', np.nan)
test.isnull().sum()

test[na_cols]= test[na_cols].astype(str).astype(float)
test[missing_cols]= test[missing_cols].astype(str).astype(float)

test['mvar47']= le.fit_transform(test['mvar47'])
test[missing_cols]= test[missing_cols].fillna(0)
test= test.fillna(test.median())
test.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,350053,1753.0,0.5001,0.000,0.0,0.0,206.0,19179.0,206.0,14221.0,...,1.0,0.0,0.000,20.551,0.42105,3.0,0.85661,0.0,0.0,1
1,350054,1894.0,1.9701,0.000,0.0,0.0,297.0,19820.0,19820.0,176895.0,...,3.0,0.0,0.000,63.047,0.00000,15.0,0.94391,0.0,0.0,0
2,350055,1774.0,0.1718,0.000,0.0,0.0,197.0,2563.0,877.0,8869.0,...,1.0,0.0,0.000,83.797,0.33333,3.0,0.76467,0.0,0.0,0
3,350056,1586.0,0.1123,5.299,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.000,0.000,1.50000,0.0,0.73274,0.0,0.0,0
4,350057,1832.0,1.4442,0.000,0.0,0.0,179.0,7577.0,179.0,21059.0,...,3.0,0.0,96.052,66.665,0.00000,23.0,0.79190,0.0,0.0,1


In [6]:
Xtest= test.drop(['application_key'], axis=1)
#Xtest= Xtest.drop(drop, axis=1)
ids= test['application_key']

In [7]:
from sklearn.preprocessing import StandardScaler
scl= StandardScaler()

Xtest= scl.fit_transform(Xtest)
X= scl.fit_transform(X)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, shuffle=True, stratify= y)
len(X_train)

56000

In [8]:
sm = SMOTE(random_state=2)

#X_train, y_train= sm.fit_sample(X_train, y_train)
X, y= sm.fit_sample(X,y)

In [19]:
rf=RandomForestClassifier(n_estimators=200, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X_train, y_train)
y_pred= rf.predict(X_test)
predictions= rf.predict_proba(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.87      0.83      0.85     18091
          1       0.54      0.63      0.58      5909

avg / total       0.79      0.78      0.78     24000

0.776458333333


In [ ]:
pred=[]
thresh= [0.45, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9]
for t in thresh:
    pred=[]
    for i in range(len(predictions)):
        if(predictions[i,0] >= t):
            pred.append(0)
        else:
            pred.append(1)
    print(classification_report(y_test, pred))
    print(accuracy_score(y_test, pred))

In [9]:
gbm = GradientBoostingClassifier(min_samples_split=200)
gbm.fit(X, y)

rf=RandomForestClassifier(n_estimators=200, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X, y)

#ada = AdaBoostClassifier(n_estimators=200)
#ada.fit(X, y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
bgg = BaggingClassifier(base_estimator=rf)
bgg.fit(X, y)
#nn = MLPClassifier( alpha=1e-5, random_state=1)
#nn.fit(X, y)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [11]:
gbm_pr= gbm.predict_proba(Xtest)
rf_pr= rf.predict_proba(Xtest)
#ada_pr= ada.predict_proba(Xtest)
bgg_pr= bgg.predict_proba(Xtest)
#nn_pr= nn.predict_proba(Xtest)

predictions= (gbm_pr+ rf_pr+ bgg_pr)/3

In [12]:
data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp11.csv", index=False)

In [106]:
rf.feature_importances_*100


array([ 10.33359286,   5.56703851,   3.35363402,   5.67030209,
         3.073733  ,   4.35028598,   3.14864913,   3.60460751,
         2.59317156,   1.86846556,   1.96595209,   2.0151234 ,
         1.85892496,   2.93536563,   2.32088316,   1.68038091,
         3.46997194,   3.37895066,   2.36346059,   2.06366888,
         3.3944064 ,   2.98559465,   1.63613972,   1.23221988,
         2.83621196,   2.59452963,   2.04237691,   1.42948107,
         2.16793211,   4.6477085 ,   1.84629177,   2.30991   ,   3.26103496])

In [12]:
model = BaggingClassifier(base_estimator=rf)
model.fit(X_train, y_train)
y_pred= model.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.88      0.81      0.84     18091
          1       0.53      0.66      0.59      5909

avg / total       0.79      0.77      0.78     24000

0.772666666667


In [13]:
clf_NN = MLPClassifier( alpha=1e-5, random_state=1)
clf_NN.fit(X_train, y_train)
y_pred= clf_NN.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.88      0.75      0.81     18091
          1       0.48      0.69      0.56      5909

avg / total       0.78      0.74      0.75     24000

0.738041666667


# Class balance by copying

In [35]:
df1= pd.read_csv("data/training3.csv")
df1.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,...,1.0,73.780,82.547,0.08696,10.0,0.63899,NaN,0.0,0,0
1,230035,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,...,0.0,96.272,NaN,0.15385,3.0,0.53241,0.0,0.0,1,0
2,230036,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,...,0.0,115.019,NaN,0.00000,1.0,0.92665,NaN,NaN,1,0
3,230039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,230040,1836.0,0.1358,0.000,0.0,0.0,347.0,38964.0,17828.0,70729.0,...,0.0,NaN,NaN,0.00000,10.0,0.89868,0.0,0.0,1,0


In [36]:
df1= df1.fillna(df1.median())
y= df1['default_ind']
cols= ['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6','mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13',
       'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20','mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25',
       'mvar26', 'mvar27','mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34','mvar35', 'mvar36', 'mvar37',
       'mvar38', 'mvar39', 'mvar40', 'mvar41','mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47']
X= df1[cols]
le= LabelEncoder()

In [37]:

X= scl.fit_transform(X)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, shuffle=True, stratify= y)
len(X_train)

83575

In [39]:
rf=RandomForestClassifier(n_estimators=300, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X_train, y_train)
y_pred= rf.predict(X_test)
predictions= rf.predict_proba(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.90      0.80      0.85     18091
          1       0.82      0.91      0.86     17728

avg / total       0.86      0.86      0.85     35819

0.855132750775


In [40]:
predictions= rf.predict_proba(Xtest)

In [41]:
predictions= rf.predict_proba(Xtest)
data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp4.csv", index=False)

# Remove rows with more than 20 cols missing

In [122]:
cols= ['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6','mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13',
       'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20','mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25',
       'mvar26', 'mvar27','mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34','mvar35', 'mvar36', 'mvar37',
       'mvar38', 'mvar39', 'mvar40', 'mvar41','mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47']

In [123]:
df.iloc[6].isnull().sum()

13

In [124]:
ind=[]
for i in range(len(df)):
    if(df.loc[i].isnull().sum() > 15):
        ind.append(i)

In [125]:
len(ind)

13271

In [126]:
df=df.drop(df.index[ind], axis=0)

In [127]:
df=df.fillna(df.median())
y= df['default_ind']
cols= ['mvar1', 'mvar2', 'mvar3', 'mvar4', 'mvar5', 'mvar6','mvar7', 'mvar8', 'mvar9', 'mvar10', 'mvar11', 'mvar12', 'mvar13',
       'mvar14', 'mvar15', 'mvar16', 'mvar17', 'mvar18', 'mvar19', 'mvar20','mvar21', 'mvar22', 'mvar23', 'mvar24', 'mvar25',
       'mvar26', 'mvar27','mvar28', 'mvar29', 'mvar30', 'mvar31', 'mvar32', 'mvar33', 'mvar34','mvar35', 'mvar36', 'mvar37',
       'mvar38', 'mvar39', 'mvar40', 'mvar41','mvar42', 'mvar43', 'mvar44', 'mvar45', 'mvar46', 'mvar47']
X= df[cols]
le= LabelEncoder()

In [128]:
X= scl.fit_transform(X)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, shuffle=True, stratify= y)
len(X_train)

46710

In [129]:
rf=RandomForestClassifier(n_estimators=100, min_samples_leaf= 5, max_features='sqrt')
rf.fit(X_train, y_train)
y_pred= rf.predict(X_test)
predictions= rf.predict_proba(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.84      0.96      0.90     15832
          1       0.66      0.33      0.44      4187

avg / total       0.81      0.82      0.80     20019

0.824366851491


In [62]:
rf.fit(X, y)
predictions= rf.predict_proba(Xtest)
data= dict({'ids': ids, 'pred-0':predictions[:,0]})
df2= pd.DataFrame(data)
df2.to_csv("data/temp7.csv", index=False)